## Transfer Learning

### Install Libraries

In [ ]:
# Importing sys to check the current Python executable
import sys
print(sys.executable)

# Import necessary libraries and install if they are not already installed
import subprocess

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# List of required packages
required_packages = [
    'numpy', 'pandas', 'opencv-python', 'Pillow', 'torch', 'torchvision',
    'scikit-learn', 'jupyter'
]

# Install the required packages
for package in required_packages:
    try:
        __import__(package)
    except ImportError:
        install(package)

# Now you can safely import the packages
import numpy as np
import pandas as pd
import os
import cv2
from PIL import Image
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score
import time # for random seed
print("All packages are installed and imported successfully!")


## Load the training data

### Get the id-labels mapping in the dataset

In [ ]:
# tiny-imagenet-200 dataset
data_dir = "tiny-imagenet-200"
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')
test_dir = os.path.join(data_dir, 'test')



# get all of the classes whose images are in the training set
training_class_ids = os.listdir(train_dir)

# class id - class label mapping
id_class_label_map = {}
with open(os.path.join(data_dir, 'words.txt'), 'r') as f:
    for line in f.readlines():
        line = line.strip().split('\t')
        class_id = line[0]
        class_label = line[1]
        id_class_label_map[class_id] = class_label


# for each class in the tiny-imagenet-200 dataset, get the class label
training_id_class_map = {}
for class_id in training_class_ids:
    class_label = id_class_label_map[class_id]
    training_id_class_map[class_id] = class_label

# we have 200 classes in the tiny-imagenet-200 dataset with their class labels

### Get random 10 classes


In [ ]:

def get_ten_random_class_ids(training_class_ids):
    # get 10 random classes

    # seed random with time
    np.random.seed(int(time.time()))

    ten_random_classes = np.random.choice(training_class_ids, 10, replace=False)
    return ten_random_classes


print("10 random classes:")
ten_random_classes = get_ten_random_class_ids(training_class_ids)
for i in range(10):
    print(f"{i+1}. {ten_random_classes[i]} - {training_id_class_map[ten_random_classes[i]]}")

### Load images of the classes

In [ ]:
# get the images in the training set for each of the 10 random classes
# class-id to class images
training_class_images = {}
for class_id in ten_random_classes:
    class_images = os.listdir(os.path.join(train_dir, class_id, 'images'))
    training_class_images[class_id] = class_images

for class_id in ten_random_classes:
    print(f"{class_id} - {training_id_class_map[class_id]} has {len(training_class_images[class_id])} images.")